In [1]:
import sqlite3
import json
from datetime import datetime

In [2]:
timeframe = '2009-05'
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY,
    comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT) """)
    
    
def format_data(data):
    data = data.replace("\n","newlinechar").replace("\r","newlinechar").replace(' " '," ' ")
    return data



def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 100:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = {}, comment_id = {}, parent = {}, comment = {}, subreddit = {}, unix = {}, score = {} WHERE parent_id ={};""".format
        (parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s_replace insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s_parent insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s_no_parent insertion',str(e))

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True



def find_parent(pid):
    try:
        sql = " SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
        
    except Exception as e:
        return False
    

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False    
    
    
    
   
if __name__ == "__main__":
    create_table()
    row_counter = 0
    paired_rows = 0
    replace_rows = 0
    no_parent = 0
    
    #with open("E:/Downloads/RC_2005-12") as f:
    with open("E:/Downloads/RC_{}".format(timeframe.split('_')[0], timeframe), buffering = 1000 ) as f:
        for row in f:
            #print(row)
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id'].split('_')[1]
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            comment_id = row['id']
            #link_id = row['link_id']
            subreddit = row['subreddit']
            parent_data = find_parent(parent_id)
            #print(parent_data)
            if score >= 2:
                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if acceptable(body):
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            replace_rows +=1
                            
                            
                else:
                    if acceptable(body):
                        if parent_data:
                            sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                            no_parent +=1
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Replace_rows:{}, Time: {}'.format(row_counter, paired_rows, replace_rows, str(datetime.now())))


Total Rows Read: 100000, Paired Rows: 4483, Replace_rows:3114, Time: 2018-06-27 15:37:50.385591
Total Rows Read: 200000, Paired Rows: 9353, Replace_rows:6217, Time: 2018-06-27 15:39:34.637554
Total Rows Read: 300000, Paired Rows: 14472, Replace_rows:9529, Time: 2018-06-27 15:41:19.896574
Total Rows Read: 400000, Paired Rows: 19275, Replace_rows:12945, Time: 2018-06-27 15:43:08.269773
Total Rows Read: 500000, Paired Rows: 24220, Replace_rows:16041, Time: 2018-06-27 15:44:55.627913
Total Rows Read: 600000, Paired Rows: 29294, Replace_rows:19441, Time: 2018-06-27 15:46:46.695266
Total Rows Read: 700000, Paired Rows: 33957, Replace_rows:22789, Time: 2018-06-27 15:48:35.025462
Total Rows Read: 800000, Paired Rows: 38846, Replace_rows:26056, Time: 2018-06-27 15:50:29.471008
Total Rows Read: 900000, Paired Rows: 43786, Replace_rows:29370, Time: 2018-06-27 15:52:23.775546
Total Rows Read: 1000000, Paired Rows: 48490, Replace_rows:32669, Time: 2018-06-27 15:54:16.358985
Total Rows Read: 1100000

In [3]:
import pandas as pd

In [4]:
timeframes = '2009-05'

for timeframe in timeframes:
    connection = sqlite3.connect('{}.db'.format(timeframes))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    curr_length = limit
    counter = 0
    test_done = False
    while curr_length == limit:
        df = pd.read_sql("SELECT * FROM parent_reply WHERE unix > {} AND parent NOT NULL AND score > 0 ORDER BY unix ASC LIMIT {} ".format(last_unix, limit), connection)
        #print(df.head(100))
        last_unix = df.tail(1)['unix'].values[0]
        cur_length = len(df)
        
        if not test_done:
            with open("test.from", 'a', encoding = 'utf8') as f:
                for content in df['parent'].values:
                    f.write(content + '\n')
                    
            with open("test.to", 'a', encoding = 'utf8') as f:
                for content in df['comment'].values:
                    f.write(content + '\n')        
            
            test_done = True
        
        else:
            with open("train.from", 'a', encoding = 'utf8') as f:
                for content in df['parent'].values:
                    f.write(content + '\n')
                    
            with open("train.to", 'a', encoding = 'utf8') as f:
                for content in df['comment'].values:
                    f.write(content + '\n')        

                        
        counter += 1
        if counter%20 == 0:
            print(counter*limit,'rows completed so far')
            
    
    
    
    
    

   parent_id comment_id                                             parent  \
0    c098nq5    c098nqv  What I love about Transformers is that it is a...   
1    c098nm5    c098nz6                                   How soon is now?   
2    c098noe    c098o37  I think the error in this study is that almost...   
3    c098nss    c098o4h                          Wow, I kind of want that.   
4    c098o28    c098o9y  Kirk HinrichnewlinecharnewlinecharMike Millern...   
5    c098nn3    c098oar                                   Satellite array.   
6    c098o4l    c098ob5                     No. I'm saying it's not theft.   
7    c098nrg    c098og4  &gt; I think it's great that they are trying t...   
8    c098o87    c098oga                 I think in perl, but dream in lisp   
9    c098o7j    c098oge  Of course, but when that sort of energy and ou...   
10   c098o1p    c098okw  They seem to be a group not defined by christi...   
11   c098odg    c098om6  If reddit would sell one of those windo

   parent_id comment_id                                             parent  \
0    c09b51b    c09b63d  You shouldn't go to jail at all.newlinecharnew...   
1    c09b4vm    c09b63j  [Statue Gropers](http://www.reddit.com/r/statu...   
2    c09b4eg    c09b64c                    One slipped past the regulators   
3    c09azqf    c09b673  He only knows that this was 5-6 years ago beca...   
4    c09ans0    c09b67c  Can somebody tell me benefits of Mint over Ubu...   
5    c09b3pl    c09b68n  ? i wasn't trying to be rude at all i just tho...   
6    c09b471    c09b68u    Something tells me he didn't walk away from it.   
7    c09b335    c09b6a9     I think the left is about as useful as cancer.   
8    c09b61b    c09b6ax  The [WaybackMachine](http://www.archive.org/we...   
9    c09b631    c09b6cv  Nobody is more hardcore than the Scots, except...   
10   c09b050    c09b6cy  I imagine it to be [henna](http://en.wikipedia...   
11   c09b5cm    c09b6d0  Why is this the top comment on this art

   parent_id comment_id                                             parent  \
0    c09dkjh    c09dmg5  Practice throwing the knife at a tree.  It wil...   
1    c09dkb9    c09dmg9                  Fuckin with me cuz I'm a teenager   
2    c09dm1u    c09dmgi     Please refrain from using such bowel language.   
3    c09dhc5    c09dmgq  The reason no one watches Scifi is because the...   
4    c09dirq    c09dmh5  Yeah anytime I use a rational argument on my d...   
5    c09dkte    c09dmhl                    I had sex in front of a cougar.   
6    c09ddfn    c09dmho  Ass. Always be prepared for a fight. A exposed...   
7    c09dlm5    c09dmjb           Yea Tommy, before zee Germans get there.   
8    c09dlm4    c09dmkf  &gt;But I've also had the reverse where I eat ...   
9    c09dgem    c09dmkt  I think Republicans would do well to drop the ...   
10   c09dlxn    c09dmky  Good start.  So what distinguishes existence f...   
11   c09dloq    c09dmm3  How did you determine peak reddit hours

   parent_id comment_id                                             parent  \
0    c09fw4t    c09fyfw     Answering*, dammit. I'm not waiting 7 minutes.   
1    c09fww3    c09fygl                 For 30% I would expect a lapdance.   
2    c09fy0l    c09fygv  If this was porno it would be an instant winne...   
3    c09fxvd    c09fyi2                                               Yes.   
4    c09fxpj    c09fyi4  I do not want this to be the ruiner of the thr...   
5    c09fvzp    c09fyio      s/Find:\s*\(.*\)\n*Replace:\s*\(.*\)/s\/\1...   
6    c09ftev    c09fyis  But by the magical law of Finders Keepers, it ...   
7    c09fxjg    c09fyj9                                    good story man.   
8    c09fua9    c09fykr                     So, kinda like Warlords, then?   
9    c09ftqi    c09fyle                                karl marx disagrees   
10   c09fmfm    c09fylg  Wow, hats off to whoever came up with this, th...   
11   c09fwed    c09fylu  &gt;I doubt it will smell worse. Nothin

   parent_id comment_id                                             parent  \
0    c09ic3c    c09icfe         Yes. Is this contradictory to what I said?   
1    c09ibwc    c09icfn  http://www.nanowerk.com/news/newsid=10517.phpn...   
2    c09i9xi    c09icfs                                  Not the Japanese.   
3    c09iajz    c09icfy  Then obviously we need a reimagining. Jesus Be...   
4    c09i71i    c09icg7  When you went to Little League games, the Duke...   
5    c09i629    c09icgx  Not to mention a book celebrating 25 years of ...   
6    c09hyl4    c09ici2                                Kind of odd for IC.   
7    c09iav1    c09icir  Apart from the Dept. of Immigration sadly.  I ...   
8    c09iayw    c09iciu  I don't have the math to back what I am going ...   
9    c09i8ek    c09icko                           Only in NZ Maori culture   
10   c09icgo    c09iclv                                            Whoosh?   
11   c09ibpa    c09icm5  Nothing surprising here ... sacrifice a

...

   parent_id comment_id                                             parent  \
0    c09ru2c    c09rut7                                     Or in the ear.   
1    c09rtty    c09rutv                   Fap fap fap fap fap fap fap fap.   
2    c09rsob    c09rutz      mothafuzuckin drought?  thats the worst kind!   
3    c09retj    c09ruu3                               This is a Valve map.   
4    c09rtkd    c09ruv5  Your attempt to start a shitty pun thread won'...   
5    c09ru2v    c09ruva                 Reminds me of the end of serenity    
6    c09rtyr    c09ruvc  Race was the factor, I'm saying it shouldn't h...   
7    c09rscj    c09ruvy  They could also use a tazer to do the pushing....   
8    c09rugf    c09ruwt                                   *venetian mutes*   
9    c09rt49    c09rux6  You should try a [Japanese Toilet](http://en.w...   
10   c09rtqw    c09ruyh  Hiding stuff from the user is bad. If there is...   
11   c09ruvl    c09ruz5               Who else would you have gi

   parent_id comment_id                                             parent  \
0    c09ubu0    c09uc39  Because only Republicans are Christian./ newli...   
1    c09u6cb    c09uc4k  &gt; I understand that they can't raise taxesn...   
2    c09u6s0    c09uc4y  He is just looking for [beatles](http://www.yo...   
3    c09u337    c09uc53                              Is this from a movie?   
4    c09ubq5    c09uc5a                              And Peru, and Spain..   
5    c09u9y6    c09uc75  there is a line.  I do believe that you've cro...   
6    c09u5jf    c09uc8m  ComputerWorld: 1998 called, it wants its layou...   
7    c09ub3g    c09uc99  On the Internet, videos are available anywhere...   
8    c09uc4d    c09uc9f                   Big Boss.  His name is Big Boss.   
9    c09uaw1    c09uccj  Blogs, man. Blogs. Web developers and designer...   
10   c09ubnl    c09uccy  I am Jewish, and I support this... on a differ...   
11   c09u572    c09ucd7  But, now... By god. Most excellent inve

   parent_id comment_id                                             parent  \
0    c09wk89    c09wnd7            Careful, he could be an American Psycho   
1    c09wmkv    c09wndd  Does anyone know if taser use incurs similar p...   
2    c09wmuf    c09wndz  No of course not but it does go to further sho...   
3    c09wmd6    c09wnev  Higher return usually equals higher risk. This...   
4    c09wk5l    c09wnfc  Scientology ad ripping off Fightclub...newline...   
5    c09wn32    c09wnfs  Nope. It means they'll use proxy servers or pe...   
6    c09wm7n    c09wngj  You left yourself wide open for a double enten...   
7    c09wcbs    c09wngn  All the gospel with half the Gnostic? I can't ...   
8    c09wlqh    c09wngp             Surely this is a *huge* underestimate.   
9    c09wdyx    c09wngw      What is La Raza? A racial pride organization?   
10   c09wmy8    c09wnhf  A.I. started out as a Kuberick movie, but Spie...   
11   c09wke3    c09wnhy  Why didn't he run for prom king? He is 

Empty DataFrame
Columns: [parent_id, comment_id, parent, comment, subreddit, unix, score]
Index: []


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
pid = "tf_c50"
pid.split('_')[1]